In [74]:
import pandas as pd
import gspread
import json
import os
import seaborn as sns
from dotenv import load_dotenv

def connect_sheets(sheet_id, sheet_name):
    """
    Realiza a conexão com a API do google sheets,
    Verifica se existem dados nos parâmetros informados
    """
    # Carrega as variáveis de ambiente
    load_dotenv()
    key_json = os.getenv('KEY_JSON')

    # Verifica se a chave JSON foi encontrada
    if not key_json:
        raise ValueError("Chave JSON não encontrada.")

    # Conexão com o Google Sheets
    google_client = gspread.service_account_from_dict(json.loads(key_json))
    worksheet = google_client.open_by_key(sheet_id).worksheet(sheet_name)

    # Obtenção dos dados e conversão para DataFrame
    dados = worksheet.get_all_values()
    if not dados:
        raise ValueError("A planilha está vazia.")

    return pd.DataFrame(dados[1:], columns=dados[0])

dados = r'../dados'
dataframes = []

# Identifica os arquivos no diretório 
arquivos = [f for f in os.listdir(dados) if f.endswith('csv')]

# Percorrendo cada arquivo do diretório
for arquivo in arquivos:
    caminho_arquivo = os.path.join(dados, arquivo)
    df = pd.read_csv(caminho_arquivo, sep=',', index_col=0)
    dataframes.append(df)

df_unificado = pd.concat(dataframes, ignore_index=True)

In [75]:
# Converte as colunas para datetime, mantendo os valores ausentes como NaT
colunas_data = ['Chegada CD', 'Início do Carregamento', 'Fim do Carregamento']
df_unificado[colunas_data] = df_unificado[colunas_data].apply(pd.to_datetime, errors='coerce')

# Extrai a data da coluna 'Chegada CD'
df_unificado['Data'] = df_unificado['Chegada CD'].dt.date

# Cria uma coluna de validação para os registros completos
df_unificado['Status'] = df_unificado['Quantidade Remessas'] > 0

In [ ]:
df_unificado.head(3)

In [77]:
tb_acompanhamento = df_unificado.groupby(['Data', 'Unidade']).agg(
                            total_registros = pd.NamedAgg(column='Status', aggfunc='count'),
                            total_valido = pd.NamedAgg(column='Status', aggfunc='sum'))

tb_acompanhamento = tb_acompanhamento.rename(columns={'total_registros': 'Total de Registros',
                                                      'total_valido': 'Registros Válidos (QTD)'})\
                                     .reset_index()

tb_acompanhamento['Registros Válidos (%)'] = tb_acompanhamento['Registros Válidos (QTD)'] / tb_acompanhamento['Total de Registros'] * 100